# Demo of Python Package:
# ROPAC - Radar Observations Processing And Compositing
This notebook serves as a demonstration, showcasing some of functionalities offered by the ROPAC Python package. The fundamental objective of the ROPAC Python package is to facilitate the efficient interpretation of radar data while concurrently introducing a sense of simplicity to complicated datasets. In this demo the goal is to demonstrate:

1. Ingesting data from various file sources.
2. Standardising the data structures.
3. Allow full control over important metadata variables.
4. Write data to the standard ODIM HDF5 format for further processing.

## 1. Let’s start by importing all the require packages and some data to explore
- For this we can use the [open-radar-data](https://pypi.org/project/open-radar-data/) package.


In [ ]:
# Import packages

import gzip
import os

from open_radar_data import DATASETS
from ropac import system_config as cfg
from ropac import PolarVolume

print(cfg.path.data)
print(cfg.path.logs)
print(cfg.path.images)
print(cfg.path.ingest)

In [ ]:
# Download Furuno SCN data
scn_filepath = DATASETS.fetch('0080_20210730_160000_01_02.scn.gz')
print(scn_filepath)

# Download Rainbow Volume data
rb5_filepath = DATASETS.fetch('2013051000000600dBZ.vol')
print(rb5_filepath)

# Download ODIM HDF5 data
odim_filepath = DATASETS.fetch('71_20181220_060628.pvol.h5')
print(odim_filepath)

- Have to unzip the FURUNO data

In [ ]:

def gunzip_file(input_path, output_path):
    with gzip.open(input_path, 'rb') as f_in, open(output_path, 'wb') as f_out:
        f_out.write(f_in.read())

# Example usage:
input_file = os.path.join(cfg.path.ingest,'0080_20210730_160000_01_02.scn.gz')
output_file = os.path.join(cfg.path.ingest,'0080_20210730_160000_01_02.scn')

gunzip_file(input_file, output_file)

print(os.listdir(cfg.path.ingest))

#update scn_filepath
scn_filepath = output_file
print(scn_filepath)

## 2. Setup CONFIG files for the radars



## 3. Read data with ROPAC

In [ ]:
scn_data = PolarVolume(scn_filepath)
print(scn_data)

1. **Radar:**
This line indicates the radar identifier of the radar. Here only the Furuno unique ID "0080" is available.

2. **Location:**
This line specifies the geographical location of the radar. It includes the longitude, latitude, and height above mean sea level (AMSL) in meters.

3. **Wavelength:**
This line states the wavelength of the radar's emitted signals. In this case, the value is "None," indicating that the wavelength information is not available.

4. **Number of Elevations:**
This line indicates the number of elevation angles or tilts used in the radar scan. In this case, there is only one elevation angle.

5. **Volume Time:**
This line shows the timestamp when the volume scan was performed, i.e., the file name timestamp.

6. **Volume Start Time:**
This line specifies the start time of the volume scan, indicating when the data collection began.

7. **Volume End Time:**
This line indicates the end time of the volume scan, representing when the data collection concluded.

8. **Available Moments:**
This line lists the different moments or variables that have been measured or calculated by the radar. The moments include parameters such as reflectivity (DBZH), Doppler velocity (VRADH), differential reflectivity (ZDR), etc. Moment names follow the ODIM naming convention.

9. **Elevation Angles:**
This line displays the specific elevation angles (tilts) used in the radar scan. In this case, the elevation angle is 7.8 degrees.

Since we did not define a config file for this radar it is missing a unique identifier based on the user’s system. If we print the radar ID name *None* is returned which will cause an error when the data is written to file as the radar ID is require in the output directory structure.

In [ ]:
print(scn_data.radar_ID )

In [ ]:
print(scn_data.get_file_path())

## 3. The PolarVolume object structure

The PolarVolume follows a similar structure as the ODIM data format. This is where the main data lives that will be written to file. Other functions are also available to increase usability.

- PolarVolume (Object)
    - dataset (list of Elevation objects)
        - data (list of Moment Objects)
            - data (2D np.array)
            - what (dataclass) metadata
        - how (dataclass) metadata
        - where (dataclass) metadata
        - what (dataclass) metadata 
    - how (dataclass) metadata
    - where (dataclass) metadata
    - what (dataclass) metadata
    
i.e., a PolarVolume is a list of Elevations, where each elevation is a List of Moments, where each Moment is a 2D numpy array of the observed data.



In [ ]:
rb5_data = PolarVolume(rb5_filepath)
print(rb5_data)

In [ ]:
# Can see from the summary output that we have 14 elevations and 1 moment.
print(len(rb5_data.dataset))
print(len(rb5_data.dataset[0].data))

In [ ]:
# We see that the first elevation in the list is 0.6 deg 
rb5_data.dataset[0].elevation

In [ ]:
# The data moment is DBZH
rb5_data.dataset[0].data[0].quantity

Lets have a look at the data:

In [ ]:
# In original unsigned interget 8-bit values as a numpy array in polar coordinates (azimuth, range)
rb5_data.dataset[0].data[0].data

In [ ]:
# Convert data to it correct units: With DBZH this will be dBZ. returns a floating point numpy array.
rb5_data.dataset[0].data[0].get_data()

In [ ]:
# Can also visualise the data with a plot
rb5_data.dataset[0].data[0].plot()

In [ ]:
# Can also take a look at the other elevations:
print(rb5_data.dataset[3].elevation)
rb5_data.dataset[3].data[0].plot()

## 4. Extract/Combine Elevations and/or Moments

In [ ]:
odim_data = PolarVolume(odim_filepath)
odim_data.radar_ID = 3 # Set radar_ID manually as this is an ODIM file with no metadata file
print(odim_data)

In [ ]:
odim_data.what.source #+ f',CMT:id=3'


#rb5_data.what.source

Here we can see we have a S-band radar from Australia that has several moments as well as 14 different elevations scans within its volume scan. 

Let’s plot the 0.5 deg elevation moments to see what the data looks like. **Each moment has an appropriate colour scale. **

In [ ]:
print(odim_data.dataset[0].elevation)
for moment in odim_data.dataset[0].data:
    moment.plot()

## 5. Visualising the scanning sequence 

In [ ]:
# Need to set radar ID as it is used within the plotting title
rb5_data.plot_scan_sequence()


In [ ]:
str(odim_data.radar_ID)



In [ ]:
odim_data.plot_scan_sequence()

Here we see both radars initiate the scanning process by employing a bottom-up elevation approach, wherein the antenna gradually moves upward with each tilt. Ideally, it is preferable to commence the scan from the top and progress downward, as the lower elevation angles typically contains the most information, and it will be more beneficial that the lower tilts be the most recent data within the scanning volume. We also observe that the scan time for the rb5_data remains relatively consistent across elevations. Conversely, the *odim_data* radar demonstrates an extended duration for scanning the bottom elevation. This characteristic indicates that the antenna rotation is slower, facilitating the acquisition of more precise data, which is what we want to see.

## 7. Writing data to file
Once you are happy with the content of the data and metadata data can be written to ODIM HDF5 file format using the write function.

In [ ]:
# the data will be automatically saved at
rb5_data.get_file_path()

In [ ]:
rb5_data.write_to_file()

In [ ]:
os.listdir(os.path.dirname(rb5_data.get_file_path()))

Data is now ready for further processing